In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import copy
import spacy
chrome_options = Options()

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

chrome_options.add_argument("--disable-notifications")

c:\Users\shrey\anaconda3\envs\ShreyanshConda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Scraping

In [11]:
# you'll need to change the location of chromedriver.exe
driver = webdriver.Chrome("C:/Users/shrey/OneDrive/Documents/chromedriver.exe", options=chrome_options)

In [8]:
# define the companies to scrape data
company_list = ['Uber','Honeywell','Intel','Verizon','Twitter','HP','Tesla','LinkedIn','Mastercard','Zoom','NVIDIA','Boeing']

# do not update this
type_1 = 'reviews'

# max pages to scrape for each company
total_pages = 10
page_list = list(range(1,total_pages+1))

In [12]:
# logging in to teamblind
# no manual input required here

driver.get('https://www.teamblind.com/company/Meta/reviews')
button_class = driver.find_element_by_class_name("signup")
button_class.find_element_by_tag_name('button').click()
login = driver.find_element_by_class_name("loginform").find_elements_by_class_name("input_wrap")
uname = login[0].find_element_by_xpath('./input')
uname.send_keys("shreyansh.agrawal@zs.com") #username
password = login[1].find_element_by_xpath('./input')
password.send_keys('myhplaptop01@') #password
submit_button = driver.find_element_by_class_name("fieldsubmit")
submit_button.click()
print("logged in")

logged in


Main code to scrape the website and export it in csv

In [ ]:
company_column = []
rating_list = []
title_review_list = []
employee_position_list = []
review_list = []
page_column = []

# after 10 secs, code will move to next page
driver.set_page_load_timeout(10)

#scraping reviews
for company in company_list:
    max_page = 10
    for page in page_list:

        if page <=max_page:

            print("good to go")
            url = 'https://www.teamblind.com/company/'+company+'/'+type_1+'?page='+str(page)
            print(company,':', page)
                
            # open the page
            try:

                delay = 20
                driver.get(url)
                try:
                    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_all')))
                    #print ("Page is ready!")
                except:
                    print ("Loading took too much time!")
                    #print("no review_all object")
                    continue

                #time.sleep(5)
                #try:
                review_all = driver.find_element_by_class_name("review_all")
                # contains the list of all reviews
                reviews = review_all.find_elements_by_class_name("review_item")

                for i in range(0, len(reviews)):

                    rating_score = reviews[i].find_element_by_class_name("rating").text
                    title_review = reviews[i].find_element_by_class_name("rvtit").text
                    employee_position = reviews[i].find_element_by_class_name("auth").text
                    review = reviews[i].find_element_by_class_name("parag").text

                    company_column.append(company)
                    page_column.append(page)

                    rating_list.append(rating_score)
                    title_review_list.append(title_review)
                    employee_position_list.append(employee_position)
                    review_list.append(review)
            # except:
                    #print("no review_all object")
                    #continue
                            
                print(len(review_list))
            
                
            except:
                #driver.execute_script("window.stop();")

                try:
                    review_all = driver.find_element_by_class_name("review_all")
                    # contains the list of all reviews
                    reviews = review_all.find_elements_by_class_name("review_item")

                    for i in range(0, len(reviews)):

                        rating_score = reviews[i].find_element_by_class_name("rating").text
                        title_review = reviews[i].find_element_by_class_name("rvtit").text
                        employee_position = reviews[i].find_element_by_class_name("auth").text
                        review = reviews[i].find_element_by_class_name("parag").text

                        company_column.append(company)
                        page_column.append(page)

                        rating_list.append(rating_score)
                        title_review_list.append(title_review)
                        employee_position_list.append(employee_position)
                        review_list.append(review)
                
                except:
                    print("page does not exist")
                    max_page = page
                    continue
                
                print(len(review_list))
        else:

            continue

        
column_list = ['Company','Page','Employee Position','Rating','Title Review','Review']
master_df = pd.DataFrame(zip(company_column,page_column,employee_position_list,rating_list,title_review_list,review_list),columns=column_list)
#driver.quit()


#writing the file
now = datetime.now()
current_time = now.strftime("%H_%M_%S")

# writes a csv file of the scraped data
master_df.to_csv("team_blind_scrape_"+current_time+".csv")

print("generated the file named: " + "team_blind_scrape_"+current_time+".csv")

In [ ]:
# compiling all the scraped data
import os
directory = 'Scraped Data'
column_list = ['Unnamed: 0','Company','Page','Employee Position','Rating','Title Review','Review']

df_scraped_raw = pd.DataFrame(columns=column_list)

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    df_scraped_raw = df_scraped_raw.append(pd.read_csv(f))

In [5]:
df_scraped_raw.to_csv("scraped_data_raw.csv")

# Run the code from here to avoid scraping

In [3]:
df_scraped_raw = pd.read_csv("scraped_data_raw.csv")
df_scraped_raw.drop(columns=['Unnamed: 0'],inplace=True)   
df_scraped_raw = df_scraped_raw.drop_duplicates()
df_scraped_raw = df_scraped_raw.reset_index()

df_review_count = df_scraped_raw['Company'].value_counts().reset_index()

# kkeping companies that have atleast 45 reviews
company_list = df_review_count[df_review_count['Company']>45]['index'].to_list()

# Data Cleaning

In [4]:
def clean_position(test):
    test = str(test)
    return test[test.rfind("*")+3:test.rfind("-")-1].strip()

def clean_rating(test):
    test = str(test)
    return test[0:test.find("0")+1]

def clean_con(test):
    test = str(test)
    return test[test.rfind("\n"):].strip()

def clean_pro(test):
    test = str(test)
    return test[test.find("\n"):test.find("Cons")].strip()

def clean_titile_review(test):
    test = str(test)
    return test[1:-1].strip()


In [5]:
df_scraped_raw['Employee Position Cleaned'] = df_scraped_raw['Employee Position'].apply(lambda x: clean_position(x))
df_scraped_raw['Rating Cleaned'] = df_scraped_raw['Rating'].apply(lambda x: clean_rating(x))
df_scraped_raw['Review_Con'] = df_scraped_raw['Review'].apply(lambda x: clean_con(x))
df_scraped_raw['Review_Pro'] = df_scraped_raw['Review'].apply(lambda x: clean_pro(x))
df_scraped_raw['Title Review Cleaned'] = df_scraped_raw['Title Review'].apply(lambda x: clean_titile_review(x))

# dropping unused columns
df_scraped_cleaned = df_scraped_raw.copy()
df_scraped_cleaned.drop(columns=['Employee Position','Rating','Review','Title Review','index'], inplace=True)
df_scraped_cleaned = df_scraped_cleaned[df_scraped_cleaned['Company'].isin(company_list)].copy()

In [6]:
# adding job roles
df_roles = pd.read_csv("job_role_mapping.csv")
df_scraped_cleaned = df_scraped_cleaned.merge(df_roles, left_on="Employee Position Cleaned",right_on="Job Role",how='left')
j_role = list(set(df_roles['Broad Job Role'].to_list()))

# Lift Analysis

In [7]:
df = df_scraped_cleaned.copy()

In [8]:
df_2 = copy.deepcopy(df)
df_2 = df_2.dropna(subset = ['Review_Pro','Review_Con'])


def sw(y):
    comment_tokens = word_tokenize(y)
    new_comment = []
    
    for i in comment_tokens:
        if i.lower() not in stopwords.words('english'):
            new_comment.append(i)
    
    return new_comment

df_2['pro_no_sw'] = df_2['Review_Pro'].apply(sw)
df_2['con_no_sw'] = df_2['Review_Con'].apply(sw)

df_2.head()

,Unnamed: 0.1,Company,Page,Employee Position Cleaned,Rating Cleaned,Review_Con,Review_Pro,Title Review Cleaned,Job Role,Broad Job Role,pro_no_sw,con_no_sw
0,0,Prudential-Financial,1,Applications Systems Architect,2.0,everything else was so terrible I did not enjo...,work life balance was truly so absolutely amaz...,good work life balance,Applications Systems Architect,Applications Developer/Engineer,"[work, life, balance, truly, absolutely, amazi...","[everything, else, terrible, enjoy, lot, micro..."
1,1,Prudential-Financial,1,Risk Management,4.0,every day is different with more changes in te...,every day is different with lots of new projec...,coping with change,Risk Management,Misc,"[every, day, different, lots, new, projects, o...","[every, day, different, changes, teams, resour..."
2,2,Prudential-Financial,1,"Manager, Consulting",3.0,limited chance of tech talent mobility and man...,Good benefits with work from home flexibility ...,WFH,"Manager, Consulting",Consulting,"[Good, benefits, work, home, flexibility, base...","[limited, chance, tech, talent, mobility, many..."
3,3,Prudential-Financial,1,Technical Analyst - Senior,4.0,It just seems to take longer for projects to g...,Great work life balance and interviews are not...,Slow moving,Technical Analyst - Senior,Analyst,"[Great, work, life, balance, interviews, bad, ...","[seems, take, longer, projects, get, approved,..."
4,4,Prudential-Financial,1,"Director, Data Scientist",3.0,Leadership does not have a sound vision for th...,You only work as much or as little as you want,"Great WLB, horrible leadership","Director, Data Scientist",Data Scientist,"[work, much, little, want]","[Leadership, sound, vision, future]"


In [13]:
word_freq_pro = df_2[['Review_Pro','pro_no_sw']].apply(pd.Series.explode).drop_duplicates()[['pro_no_sw']].value_counts().reset_index()
word_freq_pro

,pro_no_sw,0
0,.,7187
1,",",5384
2,work,5151
3,balance,3711
4,good,3602
...,...,...
10745,hesitate,1
10746,hero,1
10747,heritage,1
10748,here.lots,1


In [14]:
word_freq_con = df_2[['Review_Con','con_no_sw']].apply(pd.Series.explode).drop_duplicates()[['con_no_sw']].value_counts().reset_index()
word_freq_con

,con_no_sw,0
0,.,6883
1,",",4151
2,work,1823
3,’,1308
4,growth,1220
...,...,...
11167,hoc,1
11168,hobby,1
11169,hits,1
11170,hit/miss,1


In [15]:
pro_class = pd.read_csv('word_freq_pro_1.csv')
con_class = pd.read_csv('word_freq_con_1.csv')

In [16]:
company_ind_mapping = pd.read_csv('company_ind_mapping_1.csv')
company_ind_mapping = company_ind_mapping[['Industry','Name on Blind']]
company_ind_mapping

,Industry,Name on Blind
0,Technology,Meta
1,Consulting,IBM
2,Ecommerce,Amazon
3,Ecommerce,Wayfair
4,Technology,Microsoft
...,...,...
58,Technology,LinkedIn
59,Technology,Mastercard
60,Technology,Zoom
61,Technology,NVIDIA


# Lift - Pro Comments

In [17]:
pro_db = df_2[['Company','Review_Pro','pro_no_sw']].apply(pd.Series.explode).set_index(['Company','Review_Pro']).reset_index().drop_duplicates()[['Company','Review_Pro','pro_no_sw']]
pro_db = pro_db.merge(company_ind_mapping, how = 'left', left_on = 'Company', right_on = 'Name on Blind')
pro_db = pro_db.merge(pro_class, how = 'left', left_on = 'pro_no_sw', right_on = 'pro_no_sw')
pro_db

,Company,Review_Pro,pro_no_sw,Industry,Name on Blind,index,freq,pro_class
0,Prudential-Financial,work life balance was truly so absolutely amaz...,work,Finance,Prudential-Financial,2.0,2608.0,wlb
1,Prudential-Financial,work life balance was truly so absolutely amaz...,life,Finance,Prudential-Financial,6.0,1641.0,wlb
2,Prudential-Financial,work life balance was truly so absolutely amaz...,balance,Finance,Prudential-Financial,4.0,1788.0,wlb
3,Prudential-Financial,work life balance was truly so absolutely amaz...,truly,Finance,Prudential-Financial,509.0,26.0,NaN
4,Prudential-Financial,work life balance was truly so absolutely amaz...,absolutely,Finance,Prudential-Financial,1191.0,8.0,NaN
...,...,...,...,...,...,...,...,...
175500,IBM,Be paid with no effort for sometime(bench)\nGo...,Good,Consulting,IBM,5.0,1716.0,NaN
175501,IBM,Be paid with no effort for sometime(bench)\nGo...,name,Consulting,IBM,96.0,127.0,NaN
175502,IBM,Be paid with no effort for sometime(bench)\nGo...,mostly,Consulting,IBM,293.0,47.0,NaN
175503,IBM,Be paid with no effort for sometime(bench)\nGo...,people,Consulting,IBM,8.0,1380.0,culture


In [ ]:
#calculating the lift values
def lift(n, a, b, ab):
    l = ((n*ab)/(a*b))
    return (l)


lift_db = pro_db.copy()
lift_db = lift_db[['Company','pro_class','Review_Pro']].drop_duplicates()

lift_values = pd.DataFrame(columns=['word_1','word_2','lift_val'])

for x1 in lift_db['Company'].drop_duplicates():
    for x2 in lift_db['pro_class'].drop_duplicates():
        lift_db['x1'] = 0
        lift_db['x2'] = 0
        
        lift_db['x1'][lift_db['Company'] == x1] = 1
        lift_db['x2'][lift_db['pro_class'] == x2] = 1
        
        c = lift_db.groupby(['Review_Pro'])[['x1','x2']].sum().reset_index()   
        
        a = lift_db[lift_db['Company']==x1]['Review_Pro'].drop_duplicates().count()
        b = lift_db['x2'].sum()
        ab = c[(c['x1']>=1) & (c['x2']>=1)]['Review_Pro'].count()
        n = df['Review_Pro'].count()
        
        lift_val = lift(n, a, b, ab)
        
        app_dict = {}
        app_dict['word_1'] = x1
        app_dict['word_2'] = x2
        app_dict['lift_val'] = lift_val
        
        lift_values = lift_values.append(app_dict, ignore_index=True)
        


In [19]:
lift_values

,word_1,word_2,lift_val
0,Prudential-Financial,wlb,1.415827
1,Prudential-Financial,NaN,NaN
2,Prudential-Financial,growth,0.611356
3,Prudential-Financial,benefits,1.415455
4,Prudential-Financial,culture,1.199743
...,...,...,...
436,Lending-Club,growth,0.872511
437,Lending-Club,benefits,0.492969
438,Lending-Club,culture,0.978714
439,Lending-Club,pay,1.028357


In [20]:
similarity = pd.crosstab(lift_values['word_1'], lift_values['word_2'], lift_values['lift_val'], aggfunc=np.mean)
similarity

word_2,benefits,culture,growth,pay,remote,wlb
word_1,,,,,,
AT&T,0.972965,0.956637,0.777135,1.522239,2.601075,1.349277
Accenture,0.675977,0.885347,1.508432,0.687864,0.925600,0.956512
Adobe,1.758255,1.217955,0.724855,0.994079,0.395363,1.329636
Allstate-Financial-Services,1.920657,1.162109,0.836774,1.469082,3.080754,1.026306
Amazon,0.425820,0.891851,1.587600,1.195764,0.919450,0.731371
...,...,...,...,...,...,...
Vmware,1.333915,1.177015,0.894883,1.492126,3.255934,1.372432
Walmart,1.265286,0.919680,1.064911,1.199750,1.317878,1.059946
Wayfair,0.394375,1.155814,1.055962,1.182611,0.658939,1.166567


# Lift - Pro Comments - Sector Level

In [21]:
ind_pro_db = df_2[['Company','Review_Pro','pro_no_sw']].apply(pd.Series.explode).set_index(['Company','Review_Pro']).reset_index().drop_duplicates()[['Company','Review_Pro','pro_no_sw']]
ind_pro_db = ind_pro_db.merge(company_ind_mapping, how = 'left', left_on = 'Company', right_on = 'Name on Blind')
ind_pro_db = ind_pro_db.merge(pro_class, how = 'left', left_on = 'pro_no_sw', right_on = 'pro_no_sw')
ind_pro_db

,Company,Review_Pro,pro_no_sw,Industry,Name on Blind,index,freq,pro_class
0,Prudential-Financial,work life balance was truly so absolutely amaz...,work,Finance,Prudential-Financial,2.0,2608.0,wlb
1,Prudential-Financial,work life balance was truly so absolutely amaz...,life,Finance,Prudential-Financial,6.0,1641.0,wlb
2,Prudential-Financial,work life balance was truly so absolutely amaz...,balance,Finance,Prudential-Financial,4.0,1788.0,wlb
3,Prudential-Financial,work life balance was truly so absolutely amaz...,truly,Finance,Prudential-Financial,509.0,26.0,NaN
4,Prudential-Financial,work life balance was truly so absolutely amaz...,absolutely,Finance,Prudential-Financial,1191.0,8.0,NaN
...,...,...,...,...,...,...,...,...
175500,IBM,Be paid with no effort for sometime(bench)\nGo...,Good,Consulting,IBM,5.0,1716.0,NaN
175501,IBM,Be paid with no effort for sometime(bench)\nGo...,name,Consulting,IBM,96.0,127.0,NaN
175502,IBM,Be paid with no effort for sometime(bench)\nGo...,mostly,Consulting,IBM,293.0,47.0,NaN
175503,IBM,Be paid with no effort for sometime(bench)\nGo...,people,Consulting,IBM,8.0,1380.0,culture


In [ ]:
#calculating the lift values
def lift(n, a, b, ab):
    l = ((n*ab)/(a*b))
    return (l)


lift_db = ind_pro_db.copy()
lift_db = lift_db[['Company','Industry','pro_class','Review_Pro']].drop_duplicates()

lift_values = pd.DataFrame(columns=['word_1','word_2','lift_val'])

for x1 in lift_db['Industry'].drop_duplicates():
    for x2 in lift_db['pro_class'].drop_duplicates():
        lift_db['x1'] = 0
        lift_db['x2'] = 0
        
        lift_db['x1'][lift_db['Industry'] == x1] = 1
        lift_db['x2'][lift_db['pro_class'] == x2] = 1
        
        c = lift_db.groupby(['Review_Pro'])[['x1','x2']].sum().reset_index()   
        
        a = lift_db[lift_db['Industry']==x1]['Review_Pro'].drop_duplicates().count()
        b = lift_db['x2'].sum()
        ab = c[(c['x1']>=1) & (c['x2']>=1)]['Review_Pro'].count()
        n = df['Review_Pro'].count()
        
        lift_val = lift(n, a, b, ab)
        
        app_dict = {}
        app_dict['word_1'] = x1
        app_dict['word_2'] = x2
        app_dict['lift_val'] = lift_val
        
        lift_values = lift_values.append(app_dict, ignore_index=True)
        


In [ ]:
lift_values

In [24]:
similarity = pd.crosstab(lift_values['word_1'], lift_values['word_2'], lift_values['lift_val'], aggfunc=np.mean)
similarity

word_2,benefits,culture,growth,pay,remote,wlb
word_1,,,,,,
Consulting,0.878615,1.047641,1.437008,0.711940,0.791874,0.927945
Ecommerce,0.787038,1.092130,1.153126,1.054461,0.871543,1.151309
Finance,0.859386,1.166935,1.005139,1.110051,1.265047,1.120355
Technology,1.362744,1.081844,0.937030,1.274071,1.186031,1.140451


# Building Recommender System

In [25]:
# lowercase
df_scraped_cleaned['Review_Con'] = df_scraped_cleaned['Review_Con'].str.lower()
df_scraped_cleaned['Review_Pro'] = df_scraped_cleaned['Review_Pro'].str.lower()

In [26]:
# replacing some words

df_pro = pd.read_csv("word_freq_pro_1.csv")
df_con = pd.read_csv("word_freq_con_1.csv")

df_pro = df_pro[['pro_no_sw','pro_class']].copy()
df_con = df_con[['con_no_sw','con_class']].copy()

df_pro = df_pro[~df_pro['pro_class'].isna()].copy()
df_con = df_con[~df_con['con_class'].isna()].copy()

df_con['con_no_sw'] = df_con['con_no_sw'].str.lower()
df_pro['pro_no_sw'] = df_pro['pro_no_sw'].str.lower()

df_con['con_class'] = df_con['con_class'].str.lower()
df_pro['pro_class'] = df_pro['pro_class'].str.lower()

pro_original_word = df_pro['pro_no_sw'].to_list()
pro_replaced_word = df_pro['pro_class'].to_list()

con_original_word = df_con['con_no_sw'].to_list()
con_replaced_word = df_con['con_class'].to_list()

In [27]:
def replace_pro(text):
    for i in range(len(pro_original_word)):
        return text.replace(pro_original_word[i],pro_replaced_word[i])


def replace_con(text):
    for i in range(len(con_original_word)):
        return text.replace(con_original_word[i],con_replaced_word[i])

df_scraped_cleaned['Review_Pro'] = df_scraped_cleaned['Review_Pro'].apply(lambda x: replace_pro(x))
df_scraped_cleaned['Review_Con'] = df_scraped_cleaned['Review_Con'].apply(lambda x: replace_pro(x))
df_scraped_cleaned

,Unnamed: 0.1,Company,Page,Employee Position Cleaned,Rating Cleaned,Review_Con,Review_Pro,Title Review Cleaned,Job Role,Broad Job Role
0,0,Prudential-Financial,1,Applications Systems Architect,2.0,everything else was so terrible i did not enjo...,wlb life balance was truly so absolutely amazi...,good work life balance,Applications Systems Architect,Applications Developer/Engineer
1,1,Prudential-Financial,1,Risk Management,4.0,every day is different with more changes in te...,every day is different with lots of new projec...,coping with change,Risk Management,Misc
2,2,Prudential-Financial,1,"Manager, Consulting",3.0,limited chance of tech talent mobility and man...,good benefits with wlb from home flexibility b...,WFH,"Manager, Consulting",Consulting
3,3,Prudential-Financial,1,Technical Analyst - Senior,4.0,it just seems to take longer for projects to g...,great wlb life balance and interviews are not ...,Slow moving,Technical Analyst - Senior,Analyst
4,4,Prudential-Financial,1,"Director, Data Scientist",3.0,leadership does not have a sound vision for th...,you only wlb as much or as little as you want,"Great WLB, horrible leadership","Director, Data Scientist",Data Scientist
...,...,...,...,...,...,...,...,...,...,...
15770,431,IBM,10,Software Back End Engineer,3.0,"outdated technologies, poor compensated, offic...","satisfying wlb, knowledgeable leadership, wlb ...",Good stepping stone,Software Back End Engineer,Software Engineer
15771,432,IBM,10,Software Analyst,2.0,no learning for personal growth and stuff like...,easy wlb and stuff and like that it’s alright you,Great WLB,Software Analyst,Analyst
15772,433,IBM,10,Project Manager,3.0,financial growth very poor but no problem and ...,"good culture and environment is very friendly,...",Poor WLB,Project Manager,Misc
15773,434,IBM,10,Applications Developer,5.0,no career growth. and there is not much to learn,if you wanna be free all the time as a newbie ...,Great work life balance but no growth,Applications Developer,Applications Developer/Engineer


In [28]:
import nltk
import os 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from nltk import word_tokenize, FreqDist

nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

stop_words = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
def remove_stop_words(input_text):
    text_tokens = word_tokenize(input_text)
    tokens_without_sw =' '.join([word for word in text_tokens if not word in stopwords.words('english')])
    return tokens_without_sw

In [30]:
# removing stop words
df_scraped_cleaned['Review_Pro_wo_stopwords'] = df_scraped_cleaned['Review_Pro'].apply(remove_stop_words)
df_scraped_cleaned['Review_Con_wo_stopwords'] = df_scraped_cleaned['Review_Con'].apply(remove_stop_words)
df_scraped_cleaned.head()

,Unnamed: 0.1,Company,Page,Employee Position Cleaned,Rating Cleaned,Review_Con,Review_Pro,Title Review Cleaned,Job Role,Broad Job Role,Review_Pro_wo_stopwords,Review_Con_wo_stopwords
0,0,Prudential-Financial,1,Applications Systems Architect,2.0,everything else was so terrible i did not enjo...,wlb life balance was truly so absolutely amazi...,good work life balance,Applications Systems Architect,Applications Developer/Engineer,wlb life balance truly absolutely amazing cool,everything else terrible enjoy lot micromanage...
1,1,Prudential-Financial,1,Risk Management,4.0,every day is different with more changes in te...,every day is different with lots of new projec...,coping with change,Risk Management,Misc,every day different lots new projects opportun...,every day different changes teams resources
2,2,Prudential-Financial,1,"Manager, Consulting",3.0,limited chance of tech talent mobility and man...,good benefits with wlb from home flexibility b...,WFH,"Manager, Consulting",Consulting,good benefits wlb home flexibility based team .,limited chance tech talent mobility many progr...
3,3,Prudential-Financial,1,Technical Analyst - Senior,4.0,it just seems to take longer for projects to g...,great wlb life balance and interviews are not ...,Slow moving,Technical Analyst - Senior,Analyst,great wlb life balance interviews bad . virtua...,seems take longer projects get approved result...
4,4,Prudential-Financial,1,"Director, Data Scientist",3.0,leadership does not have a sound vision for th...,you only wlb as much or as little as you want,"Great WLB, horrible leadership","Director, Data Scientist",Data Scientist,wlb much little want,leadership sound vision future


In [32]:
import spacy
nlp = spacy.load('en_core_web_lg')
count_vectorizer = CountVectorizer()
df_similarity = df_scraped_cleaned.copy()
# can add attibutes such as wlb, growth, pay, culture, benefits, remote
required_attribute_pro = ['wlb','remote','culture','benefits','growth','pay']
#job_roles = ['Consulting','Product','Data Scientist','Data Engineer','Analyst','Program Manager','Business Technology','Finance']
#job_roles = ['Consulting']
job_roles = j_role
df_similarity_2 = df_similarity[df_similarity['Broad Job Role'].isin(job_roles)].copy()

#required_attribute_con = ['fast']

attributes_nlp_pro = nlp(' '.join(required_attribute_pro))
#attributes_nlp_con = nlp(' '.join(required_attribute_con))

def similarity_word_embeddings(review, attributes_nlp):
    review_nlp = nlp(review)
    score = review_nlp.similarity(attributes_nlp)
    return score

def cosine_bow(data,required_attribute_pro):
    final_data=[data,' '.join(required_attribute_pro)]
    vector_matrix = count_vectorizer.fit_transform(final_data)
    tokens = count_vectorizer.get_feature_names()
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    return cosine_similarity_matrix[0][1]

df_similarity_2['cosine_similarity_pro_bow'] = df_similarity_2['Review_Pro'].apply(lambda x: cosine_bow(x,required_attribute_pro))
df_similarity_2['cosine_similarity_con_bow'] = df_similarity_2['Review_Con'].apply(lambda x: cosine_bow(x,required_attribute_pro))

weights = {"pro":0.5,"con":0.5}
# df_similarity['compound_similarity'] = df_similarity['cosine_similarity_pro']*weights['pro']-df_similarity['cosine_similarity_con']*weights['con']
df_similarity_2['compound_similarity_bow'] = df_similarity_2['cosine_similarity_pro_bow']*weights['pro']-df_similarity_2['cosine_similarity_con_bow']*weights['con']

#df_similarity_2.groupby('Company').mean("compound_similarity").sort_values(by='compound_similarity_bow',ascending=False)

c:\Users\shrey\anaconda3\envs\ShreyanshConda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\shrey\anaconda3\envs\ShreyanshConda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
# top companies
df_similarity_2.groupby('Company').mean("compound_similarity").sort_values(by='compound_similarity_bow',ascending=False)[:10]

,Unnamed: 0.1,Page,cosine_similarity_pro_bow,cosine_similarity_con_bow,compound_similarity_bow
Company,,,,,
Vmware,128.047619,4.797619,0.182260,0.035085,0.073587
Splunk,93.021053,3.194737,0.160369,0.017902,0.071234
Twitter,1015.507874,5.366142,0.151273,0.019099,0.066087
LinkedIn,1683.772436,4.900641,0.161617,0.043175,0.059221
Prudential-Financial,50.000000,2.217822,0.146362,0.028574,0.058894
Adobe,143.955823,5.028112,0.154855,0.038289,0.058283
Salesforce,109.905000,3.985000,0.160542,0.044383,0.058079
Dell,315.500000,5.490000,0.164389,0.055072,0.054658
Silicon-Valley-Bank,203.666667,1.518519,0.153979,0.045257,0.054361


In [35]:
# worst companies
df_similarity_2.groupby('Company').mean("compound_similarity").sort_values(by='compound_similarity_bow',ascending=True)[:10]

,Unnamed: 0.1,Page,cosine_similarity_pro_bow,cosine_similarity_con_bow,compound_similarity_bow
Company,,,,,
Tesla,1396.000000,5.368421,0.056060,0.085858,-0.014899
ByteDance,168.501916,5.498084,0.068605,0.063681,0.002462
Infosys,476.319820,5.662162,0.068248,0.063036,0.002606
Amazon,555.793333,5.476667,0.075497,0.065816,0.004841
PwC,1364.500000,5.500000,0.085768,0.072864,0.006452
McKinsey,1073.500000,5.212766,0.088446,0.072328,0.008059
Truist,27.600000,1.488889,0.106198,0.082649,0.011775
Broadcom,111.500000,4.250000,0.082103,0.056872,0.012615
KPMG,741.500000,5.246479,0.090000,0.064176,0.012912
